>80/20 rule:
The Pareto principle (also known as the 80/20 rule, the law of the vital few, or the principle of factor sparsity)[1][2] states that, for many events, roughly 80% of the effects come from 20% of the causes
the dataset given is based on the above rule

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys
import math
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
from pandas.io.json import json_normalize
from datetime import datetime

import os
print('done')
print(os.listdir("../input"))


The data is in CSV file format. lets view the data to understand it better.

In [ ]:
#data_train = pd.read_csv("../input/train_v2.csv",sep=',')
train_data = pd.read_csv('../input/train_v2.csv',sep=',',nrows=2000)
display(train_data.head())

we can observe from the above data that four columns device, geoNetwork, totals and trafficSource are in json format.We will first flatten this data. we will be using the following function given by julian

In [ ]:
def load_df(csv_path='../input/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows,usecols=lambda col: col not in ["hits","customDimensions"]
)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../input"))


now we can easily load both the training and the testing dataset

In [ ]:
%%time
train_data = load_df("../input/train_v2.csv",nrows=1000000)
test_data = load_df("../input/test_v2.csv")

In [ ]:
display(train_data.head())

In [ ]:
display(test_data.head(n=20))

Before starting with analysis it is good to first observe the data and make some derivations
I have observed the following in the above dataset:
1. many columns have the value "not available in demo dataset" which will not be useful.
2. visitId and visitStartTime is the same
3.. date field is a combination of year, month and date

In [ ]:
print(train_data.shape,test_data.shape)

we can modify the date field to obtain it in proper date format, we can also make seperate columns for date, month as well as day. the following code can be used to do it.

In [ ]:
def process_date(data):
    data['date'] = data['date'].astype(str)
    data["date"] = data["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    data["date"] = pd.to_datetime(data["date"])   
    data["month"] = data['date'].dt.month
    data["day"] = data['date'].dt.day
    data["weekday"] =data['date'].dt.weekday 
    return data

In [ ]:
train_data = process_date(train_data)
test_data = process_date(test_data)
display(train_data.head())
display(test_data.head())


In [ ]:
print(test_data.shape)
print(train_data.shape)

Now we will have to explore the dataset, lets first find the data type of each column

In [ ]:
print(train_data.info())


The above result shows that we have 1 boolean type, 5 integer type 54 object type and 1 date time type of data. we have a lot of null values in our dataset, lets find a count of  all the null type of objects

In [ ]:
null_cnt = train_data.isnull().sum().sort_values()
print(null_cnt[null_cnt > 0])


We have to first fill all the null values in the dataset.
we have three different kinds of data
1. object
2. numeric
3. boolean
one by one we have to fill the null values.
by checking manually, i divided all the columns having null values into the above three categories.

numerical : totals.pageviews  , totals.newVisits, totals.bounces, totals.timeOnSite, totals.sessionQualityDim, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue

object : trafficSource.keyword, trafficSource.referralPath, trafficSource.adwordsClickInfo.gclId, trafficSource.adwordsClickInfo.isVideoAd, trafficSource.adwordsClickInfo.page, trafficSource.adwordsClickInfo.slot, trafficSource.adwordsClickInfo.adNetworkType, trafficSource.adContent   
boolean: trafficSource.isTrueDirect        

now we will deal with each of these null value columns.


In [ ]:
# for object type column we can just fill not there in place of null
for column in ['trafficSource.keyword',
            'trafficSource.referralPath',
            'trafficSource.adwordsClickInfo.gclId',
            'trafficSource.adwordsClickInfo.adNetworkType',
            'trafficSource.adwordsClickInfo.isVideoAd',
            'trafficSource.adwordsClickInfo.page',
            'trafficSource.adwordsClickInfo.slot',
            'trafficSource.adContent','trafficSource.isTrueDirect']:
    train_data[column].fillna('not_there', inplace=True)
    test_data[column].fillna('not_there', inplace=True)
    
    
# for boolean type column we fill false in place of Nan
train_data['trafficSource.isTrueDirect'].fillna(False, inplace=True)
test_data['trafficSource.isTrueDirect'].fillna(False, inplace=True)

# in numeric columns we do the following changes keeping the respective columns in mind
train_data['totals.pageviews'].fillna(1, inplace=True)
train_data['totals.newVisits'].fillna(0, inplace=True)
train_data['totals.bounces'].fillna(0, inplace=True)
train_data['totals.timeOnSite'].fillna(0, inplace=True)
train_data['totals.sessionQualityDim'].fillna(0, inplace=True)
train_data['totals.transactions'].fillna(0, inplace=True)
train_data['totals.transactionRevenue'].fillna(0, inplace=True)
train_data['totals.totalTransactionRevenue'].fillna(0, inplace=True) 

test_data['totals.pageviews'].fillna(1, inplace=True)
test_data['totals.newVisits'].fillna(0, inplace=True)
test_data['totals.bounces'].fillna(0, inplace=True)
test_data['totals.timeOnSite'].fillna(0, inplace=True)
test_data['totals.sessionQualityDim'].fillna(0, inplace=True)
test_data['totals.transactions'].fillna(0, inplace=True)
test_data['totals.transactionRevenue'].fillna(0, inplace=True)
test_data['totals.totalTransactionRevenue'].fillna(0, inplace=True)
    


as observed, there are few columns having the constant value "not available in demo dataset" we can ignore these columns. It will make predictions easier and complutationally efficient. there are few other columns with constant data type as well, these columns will not make any difference in our analysis,  so we delete these columns in the following code.

In [ ]:

drop_column = [column for column in train_data.columns if train_data[column].nunique() == 1]
drop_column1 = [column for column in test_data.columns if train_data[column].nunique() == 1]
for c in drop_column:
    print(c + ':', train_data[c].unique())

print('droped columns are:', drop_column)
final_train=train_data.drop(drop_column, axis=1)
final_test=test_data.drop(drop_column1, axis=1)


#final_train = train_data.loc[:, (train_data != train_data.iloc[0]).any()]
#final_test = test_data.loc[:, (test_data != test_data.iloc[0]).any()]


Now if we see the size of the data sets we can observe that it is reduced.

In [ ]:
print(final_train.shape)
print(final_test.shape)

In [ ]:
print(final_train.info())

**Now our data is ready for the analysis, we can start with the analysis and visualization.**


The variable to be predicted is transaction revenue.
so we first analysize this feature.
As we are predicting the natural log of sum of all transactions of the user, we code for the same and then take  a scatter plot.

In [ ]:
final_train["totals.transactionRevenue"] = final_train["totals.transactionRevenue"].astype('float')
plot1 = final_train.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

plt.figure(figsize=(10,5))
plt.scatter(range(plot1.shape[0]), np.sort(np.log1p(plot1["totals.transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

Thus as per the 80/20 rule mentioned in the problem statement, very less amount of customers are contributing for the revenue being generated.

Nows lets analyse how the different attributes affect the revenue.
we first start with date.

In [ ]:

def scatter_plot(var, color):
    trace = go.Scatter(
        x=var.index[::-1],
        y=var.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace


var = final_train.groupby('date')['totals.transactionRevenue'].agg(['size', 'count'])
var.columns = ["total count", "non-zero revenue count"]
var = var.sort_index()
#cnt_srs.index = cnt_srs.index.astype('str')
trace1 = scatter_plot(var["total count"], 'grey')
trace2 = scatter_plot(var["non-zero revenue count"], 'orange')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["Date - total Count", "Date - Non-zero Revenue count"])
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, title="plot of date against revenue counts")
py.iplot(fig, filename='date vs revenue counts')

from the above graph we can see that there is no particular pattern between the revenue generated and the dates. 
we can notice a very high peak during end of the year 2017, except for that there are uniform peaks everywhere.
but there is a possibility of finding a pattern if we analyse the data weekly as there are uniform peaks .

In [ ]:

def scatter_plot(var, color):
    trace = go.Scatter(
        x=var.index[::-1],
        y=var.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace


var = final_train.groupby('weekday')['totals.transactionRevenue'].agg(['size', 'count'])
var.columns = ["total count", "non-zero revenue count"]
var = var.sort_index()
#cnt_srs.index = cnt_srs.index.astype('str')
trace1 = scatter_plot(var["total count"], 'grey')
trace2 = scatter_plot(var["non-zero revenue count"], 'orange')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["weekday - total Count", "weekday - Non-zero Revenue count"])
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, title="plot of weekday against revenue counts")
py.iplot(fig, filename='weekday vs revenue counts')

From the above analysis we can see a pattern, the revenue count is high during the second and third day of the week and falls down after it during weekend.
the revenue count is highest on wednesdays.


We can next analyse channel grouping.

In [ ]:

channel_group = final_train['channelGrouping'].value_counts()
channel_group.plot(kind='bar',rot=20, title= 'Channel Distribution ',figsize=(14,5))
plt.show()



We can clearly see that organic search has the highest count. but we need to find out relation between channel and revenue .
for doing it we will group revenue by channel and see the result

In [ ]:
train_rev = final_train[final_train['totals.transactionRevenue'] > 0].copy()
def nonzeroBar(a, b, colName, topN=np.nan):
    df1=b[colName].value_counts()[:10]
    df1.plot(kind='bar',color="brown", title= colName+' for zero revenue ',figsize=(14,5))
    

In [ ]:
nonzeroBar(final_train,train_rev, 'channelGrouping')

From the above analysis we can see that even though organic search has the highest count,but  the revenue generated is more through referral.


We do the same analysis for device browser now

In [ ]:
channel_group = final_train['device.browser'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'device browser Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'device.browser')



* We can see that chrome and safari have the highest count as well as are the two highest revenue generating browsers. 

In [ ]:
channel_group = final_train['device.deviceCategory'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'device category Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'device.deviceCategory')

From the above graphs it is clearly seen that desktop is the highest revenue generator . 
mobile has a hight count but doesnt generate that much revenue.



In [ ]:
channel_group = final_train['device.operatingSystem'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'device operating System Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'device.operatingSystem')

In [ ]:

channel_group = final_train['geoNetwork.city'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'geo Network city Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'geoNetwork.city')


Here mountain view has highest count but new york generates the most amount of revenue from among the columns known.

In [ ]:

channel_group = final_train['geoNetwork.region'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'geo Network region Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'geoNetwork.region')



clearly we can see that california generates the highest revenue.

In [ ]:

channel_group = final_train['geoNetwork.continent'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'geo Network continent Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'geoNetwork.continent')


Among continents america generates the highest revenue

In [ ]:


channel_group = final_train['visitNumber'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= 'Visit Nuber Distribution ',figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'visitNumber')




more the number of visits lesser is the probability of the person generating revenue

In [ ]:



channel_group = final_train['totals.timeOnSite'].value_counts()[:10]
channel_group.plot(kind='bar',rot=20, title= "time on site Distribution",figsize=(14,5))
plt.show()

nonzeroBar(final_train,train_rev,'totals.timeOnSite')





time on site doesnt matter

In [ ]:
# label encoding
cols = ["channelGrouping", "device.browser", "device.deviceCategory", "device.operatingSystem", "geoNetwork.city",
        "geoNetwork.continent", "geoNetwork.country", "geoNetwork.metro","geoNetwork.networkDomain","geoNetwork.region",
        "geoNetwork.subContinent","trafficSource.adContent","trafficSource.adwordsClickInfo.adNetworkType", 
        "trafficSource.adwordsClickInfo.gclId",  "trafficSource.adwordsClickInfo.page", 
        "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign","trafficSource.keyword",
        "trafficSource.medium", "trafficSource.source",'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(final_train[col].values.astype('str')) + list(final_test[col].values.astype('str')))
    final_train[col] = lbl.transform(list(final_train[col].values.astype('str')))
    final_test[col] = lbl.transform(list(final_test[col].values.astype('str')))


In [ ]:
#grouping all numerical columns and converting the m to float
cols1 = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits','totals.timeOnSite','totals.sessionQualityDim','totals.transactions','totals.totalTransactionRevenue']    
for col in cols1:
    final_train[col] = final_train[col].astype(float)
    final_test[col] = final_test[col].astype(float)


In [ ]:
train_x = final_train[cols + cols1] 
valid_x = final_test[cols + cols1]  

final_train["totals.transactionRevenue"] = np.log1p(final_train["totals.transactionRevenue"].astype(float))
final_test["totals.transactionRevenue"] = np.log1p(final_test["totals.transactionRevenue"].astype(float))
train_y = final_train["totals.transactionRevenue"]
valid_y=final_test["totals.transactionRevenue"]

In [ ]:
#lgb function
def func_lgb(train_x, train_y, val_x, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgvalue = lgb.Dataset(valid_x, label=valid_y)
    model = lgb.train(params, lgtrain,1000, valid_sets=[lgvalue], early_stopping_rounds=80, verbose_eval=100)
    
    predicted_val_y = model.predict(val_x, num_iteration=model.best_iteration)
    return  model, predicted_val_y

# Training the model #
model, predicted_val = func_lgb(train_x, train_y, valid_x, valid_y)

In [ ]:
submission = pd.DataFrame({"fullVisitorId":final_test["fullVisitorId"].values})
predicted_val[predicted_val<0] = 0
submission["PredictedLogRevenue"] = np.expm1(predicted_val)
submission = submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
submission.columns = ["fullVisitorId", "PredictedLogRevenue"]
submission["PredictedLogRevenue"] = np.log1p(submission["PredictedLogRevenue"])


In [ ]:
submission.head()

In [ ]:
submission.to_csv("submit.csv", index=False)